In [7]:
# Exploring the input data, doing things like counting proportions of each class

import pandas as pd

multi_train_path = "/Users/dnsosa/Desktop/AltmanLab/ContradictoryClaims/drug-lit-contradictory-claims/input/multinli/multinli_1.0_train.txt"
multi_test_path = "/Users/dnsosa/Desktop/AltmanLab/ContradictoryClaims/drug-lit-contradictory-claims/input/multinli-dev/multinli_1.0_dev_matched.txt"

downsample = 0.1

multinli_train_data = pd.read_csv(multi_train_path, sep='\t', error_bad_lines=False).dropna(subset=["sentence1", "sentence2"]).sample(frac=downsample)
multinli_test_data = pd.read_csv(multi_test_path, sep='\t', error_bad_lines=False).dropna(subset=["sentence1", "sentence2"]).sample(frac=1)




b'Skipping line 24810: expected 15 fields, saw 16\nSkipping line 33961: expected 15 fields, saw 16\n'
b'Skipping line 75911: expected 15 fields, saw 16\nSkipping line 100114: expected 15 fields, saw 16\n'
b'Skipping line 150638: expected 15 fields, saw 16\nSkipping line 158834: expected 15 fields, saw 16\nSkipping line 173104: expected 15 fields, saw 16\nSkipping line 178252: expected 15 fields, saw 16\n'
b'Skipping line 221951: expected 15 fields, saw 16\n'
b'Skipping line 286845: expected 15 fields, saw 16\nSkipping line 314110: expected 15 fields, saw 16\n'
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
print(len(multinli_train_data[multinli_train_data.gold_label == "neutral"]))
print(len(multinli_train_data[multinli_train_data.gold_label == "contradiction"]))
print(len(multinli_train_data[multinli_train_data.gold_label == "entailment"]))

12948
13131
13033


In [15]:
print(len(multinli_test_data[multinli_test_data.gold_label == "neutral"]))
print(len(multinli_test_data[multinli_test_data.gold_label == "contradiction"]))
print(len(multinli_test_data[multinli_test_data.gold_label == "entailment"]))

3094
3180
3440


In [1]:
from contradictory_claims.data.make_dataset import create_mancon_sent_pairs_from_xml,\
    load_cord_pairs_v2, load_drug_virus_lexicons, load_mancon_corpus_from_sent_pairs,\
    load_med_nli, load_multi_nli

import os

root_dir = "/Users/dnsosa/Desktop/AltmanLab/ContradictoryClaims/drug-lit-contradictory-claims/"

mednli_train_path = os.path.join(root_dir, 'input/mednli/mli_train_v1.jsonl')
mednli_dev_path = os.path.join(root_dir, 'input/mednli/mli_dev_v1.jsonl')
mednli_test_path = os.path.join(root_dir, 'input/mednli/mli_test_v1.jsonl')

drug_lex_path = os.path.join(root_dir, 'input/drugnames/DrugNames.txt')
virus_lex_path = os.path.join(root_dir, 'input/virus-words/virus_words.txt')

drug_names, virus_names = load_drug_virus_lexicons(drug_lex_path, virus_lex_path)

med_nli_train_x, med_nli_train_y, med_nli_test_x, med_nli_test_y = \
            load_med_nli(mednli_train_path, mednli_dev_path, mednli_test_path, multi_class=True,
                         drug_names=drug_names)

10451 unique drugs found in training & testing corpus:


In [24]:
print(med_nli_train_y.sum(axis=0))
print(med_nli_test_y.sum(axis=0))

[4209. 4209. 4209.]
[474. 474. 474.]


In [2]:
# ManConCorpus xml path
mancon_xml_path = os.path.join(root_dir, 'input/manconcorpus/ManConCorpus.xml')

# ManConCorpus processed path
mancon_sent_pairs = os.path.join(root_dir, 'input/manconcorpus-sent-pairs/manconcorpus_sent_pairs_v2.tsv')
mancon_eval_path = os.path.join(root_dir, 'input/manconcorpus/manconcorpus_benchmark_eval_data.tsv')

# Create ManCon sentence pairs dataset. Also return the eval data to be used
mancon_eval_data = create_mancon_sent_pairs_from_xml(mancon_xml_path, mancon_sent_pairs, mancon_eval_path)

man_con_train_x, man_con_train_y, man_con_test_x, man_con_test_y = \
            load_mancon_corpus_from_sent_pairs(mancon_sent_pairs, multi_class=True, drug_names=drug_names)

Number of contradiction pairs: 726
Number of entailment pairs: 726
Number of neutral pairs: 726


In [31]:
man_con_test_y.sum(axis=0)

array([12259.,   210.,   145.], dtype=float32)

In [8]:
mancon_data = pd.read_csv(mancon_sent_pairs, sep='\t')
mancon_data['label'] = [2 if label == 'contradiction' else 1 if label == 'entailment' else 0 for
                        label in mancon_data.label]

In [9]:
g = mancon_data.groupby('label', group_keys=False)
mancon_data = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()))).reset_index(drop=True)
print(len(mancon_data[mancon_data.label == 0]))
print(len(mancon_data[mancon_data.label == 1]))
print(len(mancon_data[mancon_data.label == 2]))

726
726
726


In [10]:
mancon_data

,text_a,text_b,label,guid
0,The results suggest that genetic factors play ...,In the treatment of patients with hypertension...,0,56764
1,Synergism of angiotensin-converting enzyme I/D...,Among patients with chronic ischemic heart fai...,0,2243
2,"If replicated, LGE may be considered an import...",Daily electronic transmission of body weight a...,0,10417
3,Losartan conferred significant renal benefits ...,"In this population, moderate intake of fatty f...",0,23080
4,It is concluded that the C242T polymorphism of...,Captopril protects against deterioration in re...,0,51114
...,...,...,...,...
2173,The results indicate that total treatment cost...,Home telecare may not offer incremental benefi...,2,236
2174,Captopril protects against deterioration in re...,Quinapril 20 mg did not significantly affect t...,2,126
2175,The ecNOS gene a/b polymorphism and the NADH/N...,In the total population and in high and low ri...,2,590
2176,A formal education and support intervention su...,Telemedicine management may not be appropriate...,2,226


In [ ]:
# now to save and load model -- Biobert


trained_model, train_history = train_model(multi_nli_train_x, multi_nli_train_y,
                                           multi_nli_test_x, multi_nli_test_y,
                                           med_nli_train_x, med_nli_train_y,
                                           med_nli_test_x, med_nli_test_y,
                                           man_con_train_x, man_con_train_y,
                                           man_con_test_x, man_con_test_y,
                                           cord_train_x, cord_train_y,
                                           cord_test_x, cord_test_y,
                                           drug_names, virus_names,
                                           model_name="allenai/biomed_roberta_base",
                                           use_multi_nli=use_multinli,
                                           use_med_nli=use_mednli,
                                           use_man_con=use_mancon,
                                           use_cord=use_roamdev,
                                           epochs=epochs,
                                           batch_size=batch_size,  # class_weights, aux_input,
                                           learning_rate=learning_rate,
                                           multi_class=multi_class)

# Save model
# out_dir = 'output/working/biomed_roberta_base/'
# out_dir = output_dir
# biobert_out_dir = f"output/trained_biobert/biobert_{uid}"
save_model(trained_model, timed_dir_name=False, transformer_dir=out_dir)
shutil.make_archive('biobert_output', 'zip', root_dir=out_dir)  # ok currently this seems to do nothing

